In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sb 
from sklearn.compose import make_column_transformer

# les méthodes de prétraitement

from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder,OrdinalEncoder

# les models de machine learning pour la situation

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  confusion_matrix, classification_report

#from xgboost import XGBCClassifier
#import pip
#pip.main(['install','xgboost'])
import warnings 
warnings.filterwarnings('ignore')

- CHARGEMENT DU JEU DE DONNNEES

In [2]:
df=pd.read_csv("dataset.csv",sep=";",na_values="Unknown")

In [3]:
df=df.drop('CLIENTNUM',axis=1)
df

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,NaN,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,3,2,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462
10123,Attrited Customer,41,M,2,NaN,Divorced,$40K - $60K,Blue,25,4,2,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511
10124,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,5,3,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000
10125,Attrited Customer,30,M,2,Graduate,NaN,$40K - $60K,Blue,36,4,3,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000


- DATAPREPROCESSING

In [4]:
#separation du jeu de donnée

In [5]:
##target
y=df["Attrition_Flag"]

In [6]:
y.replace({'Existing Customer':0,'Attrited Customer':1},inplace=True)
df.drop("Attrition_Flag",axis=1,inplace=True)

In [7]:
df_train,df_test,y_train,y_test = train_test_split(df,y,test_size=0.3,random_state=42)

In [8]:
#Imputation dans le jeu d'entrainement
for i in list(df_test.columns)[1:-1]:
    if df_train[i].dtype == 'object':
        #imputation par le mode des colonnes qualitatives
        df_train[i].fillna(value=df_train[i].mode()[0],inplace=True) 

In [9]:
df_train.isna().sum()/df_train.shape[0]*100

Customer_Age                0.0
Gender                      0.0
Dependent_count             0.0
Education_Level             0.0
Marital_Status              0.0
Income_Category             0.0
Card_Category               0.0
Months_on_book              0.0
Total_Relationship_Count    0.0
Months_Inactive_12_mon      0.0
Contacts_Count_12_mon       0.0
Credit_Limit                0.0
Total_Revolving_Bal         0.0
Avg_Open_To_Buy             0.0
Total_Amt_Chng_Q4_Q1        0.0
Total_Trans_Amt             0.0
Total_Trans_Ct              0.0
Total_Ct_Chng_Q4_Q1         0.0
Avg_Utilization_Ratio       0.0
dtype: float64

In [10]:
X_cat = df_train.select_dtypes(object)
X_num = df_train.select_dtypes(exclude=object)

- Encodage

In [11]:
X_cat["Income_Category"].replace({"Less than $40K":0,"$40K - $60K":1,"$60K - $80K":2,"$80K - $120K":3,"$120K +":4},inplace=True)

In [12]:
X_cat["Card_Category"].replace({"Blue":0,"Silver":1,"Gold":2,"Platinum":3},inplace=True)
X_cat["Education_Level"].replace({"Uneducated":0,"College":1,"High School":2,"Graduate":3,"Post-Graduate":4,"Doctorate":5},inplace=True)

In [13]:
def categorie_encoder(data,cat_feature):
    for col in cat_feature:
        dummy_data=pd.get_dummies(data[col], dummy_na=False, drop_first=True)
        data=data.drop(col, axis='columns')
        data=pd.concat([data,dummy_data], axis='columns')
    return data

In [14]:
X_cat=categorie_encoder(X_cat,['Gender','Marital_Status'])

In [15]:
X_cat

,Education_Level,Income_Category,Card_Category,M,Married,Single
415,4,2,0,1,1,0
3749,2,0,0,0,0,1
9295,1,3,1,1,1,0
8290,3,0,0,0,1,0
7672,2,0,0,0,0,1
...,...,...,...,...,...,...
5734,3,1,0,0,0,1
5191,2,0,0,0,0,0
5390,3,0,0,0,1,0
860,3,1,0,1,0,1


In [16]:
X_num = df_train.select_dtypes(exclude=object)

In [17]:
col=['Months_Inactive_12_mon', 'Contacts_Count_12_mon',
       'Total_Revolving_Bal', 'Total_Trans_Amt', 'Total_Trans_Ct',
       'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
X_num=X_num[col]

In [18]:
X_num

,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
415,2,0,1965,1329,26,0.444,0.243
3749,2,2,0,3889,67,0.489,0.000
9295,1,3,1131,14716,112,0.750,0.033
8290,1,3,2517,4608,61,1.033,0.828
7672,3,3,0,2523,45,0.607,0.000
...,...,...,...,...,...,...,...
5734,2,5,2517,2104,44,0.833,0.608
5191,3,6,0,2269,42,0.312,0.000
5390,1,2,1547,4678,74,1.000,0.669
860,1,1,1707,1482,42,0.312,0.479


In [19]:
scaler = StandardScaler()
X_num[col] = scaler.fit_transform(X_num[col])

In [20]:
import pickle
scaler= pickle.dump(scaler,open('scaler.pkl','wb'))

In [23]:
df_train[col].loc[415,:]

Months_Inactive_12_mon       2.000
Contacts_Count_12_mon        0.000
Total_Revolving_Bal       1965.000
Total_Trans_Amt           1329.000
Total_Trans_Ct              26.000
Total_Ct_Chng_Q4_Q1          0.444
Avg_Utilization_Ratio        0.243
Name: 415, dtype: float64

In [28]:
y[415]

0

In [30]:
df_train[col].loc[10123,:]

KeyError: 10123

In [29]:
y[10123]

1